# Data Ingestion

## Copy CSV files to S3

In [ ]:
# Assume all the pre-requisites were set up 
%store -r setup_instance_check_passed
%store -r setup_dependencies_passed
%store -r setup_s3_bucket_passed
%store -r setup_iam_roles_passed

In [3]:
# Load Libraries
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
# Public Data
!aws s3 ls s3://ads508team5/

# Copy Datasets from :
# Source of data
s3_public_path_csv = "s3://ads508team5/"
%store s3_public_path_csv

# Destination Data: 
s3_private_path_csv = "s3://{}/ADS508_project/".format(bucket)
%store s3_private_path_csv

# Copy datasets
!aws s3 cp --recursive $s3_public_path_csv/ $s3_private_path_csv/ --exclude "*" --include "hashtag_donaldtrump.csv"
!aws s3 cp --recursive $s3_public_path_csv/ $s3_private_path_csv/ --exclude "*" --include "hashtag_joebiden.csv"
!aws s3 cp --recursive $s3_public_path_csv/ $s3_private_path_csv/ --exclude "*" --include "nyt-comments-2020.csv"


2024-03-14 19:04:41  483859793 hashtag_donaldtrump.csv
2024-03-14 19:04:41  380820416 hashtag_joebiden.csv
2024-03-14 18:35:45 3066945799 nyt-comments-2020.csv
Stored 's3_public_path_csv' (str)
s3://ads508team5/
s3://sagemaker-us-west-2-471112815505/ADS508_project/
Stored 's3_private_path_csv' (str)


In [9]:
# End of Copy CSV files to S3
%store

Stored variables and their in-db values:
s3_private_path_csv             -> 's3://sagemaker-us-west-2-471112815505/ADS508_proj
s3_public_path_csv              -> 's3://ads508team5/'


## Create Database and Tables
### Database

In [10]:
# Setup and check pre-requisites to create Database
ingest_create_athena_db_passed = False
%store -r s3_public_path_csv
%store -r s3_private_path_csv

!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [11]:
# Create Database
database_name = "dbpoliticpulsecomment"

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

# pd.read_sql(statement, conn)

# Verify DB successfully created
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)


/tmp/ipykernel_130/2339478186.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [ ]:
# End of Create Database
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

%store ingest_create_athena_db_passed
%store

### Tables

In [ ]:
# Setup and check pre-requisites to create Tables
ingest_create_athena_table_passed = False
%store -r ingest_create_athena_db_passed

In [6]:
# Create Tweeter Tables
table_name_csv = "hastag_donaldtrump"
s3_private_path_csv = "s3://ads508team5/hashtag_donaldtrump.csv"
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
  created_at DATETIME,
  tweet_id FLOAT,
  tweet VARCHAR(250),
  likes INT,
  retweet_count INT,
  source VARCHAR(45),
  user_id INT,
  user_name VARCHAR(250),
  user_screen_name VARCHAR(45),
  user_description VARCHAR(250),
  user_join_date DATETIME,
  user_followers_count INT,
  user_location VARCHAR(45),
  lat FLOAT,
  long FLOAT,
  city VARCHAR(45),
  country VARCHAR(45),
  continent VARCHAR(45),
  state VARCHAR(45),
  state_code VARCHAR(45),
  collected_at VARCHAR(45) 
) LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

table_name_csv = "hastag_joebiden"
s3_private_path_csv = "s3://ads508team5/hashtag_joebiden.csv"



CREATE EXTERNAL TABLE IF NOT EXISTS dbpoliticpulse2020.hastag_donaldtrump(
  created_at DATETIME,
  tweet_id FLOAT,
  tweet VARCHAR(250),
  likes INT,
  retweet_count INT,
  source VARCHAR(45),
  user_id INT,
  user_name VARCHAR(250),
  user_screen_name VARCHAR(45),
  user_description VARCHAR(250),
  user_join_date DATETIME,
  user_followers_count INT,
  user_location VARCHAR(45),
  lat FLOAT,
  long FLOAT,
  city VARCHAR(45),
  country VARCHAR(45),
  continent VARCHAR(45),
  state VARCHAR(45),
  state_code VARCHAR(45),
  collected_at VARCHAR(45) 
) LOCATION 's3://ads508team5/hashtag_donaldtrump.csv'
TBLPROPERTIES ('skip.header.line.count'='1')


In [ ]:
# Create NYT comment Table
table_name_csv = "nyt_comment"
s3_private_path_csv = "s3://ads508team5/nyt-comments-2020.csv"

statement = """CREATE TABLE {}.{} (
  commentID INT,
  status VARCHAR(20),
  commentSequence INT,
  userID INT,
  userDisplayName VARCHAR(45),
  userLocation VARCHAR(45),
  userTitle VARCHAR(10),
  commentBody VARCHAR(500),
  createDate DATETIME,
  updateDate DATETIME,
  approveDate DATETIME,
  recommendation INT,
  replyCount INT,
  editorsSelection TEXT,
  parentID INT,
  parentUserDisplayName VARCHAR(45),
  depth INT,
  commentType TEXT,
  trusted TEXT,
  recommendedFlag TEXT,
  permID INT,
  isAnonymous TEXT,
  articleID VARCHAR(150) NULL 
) LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv)

In [ ]:
# End of Create Database and Tables
%store ingest_create_athena_table_passed
%store

## Query Data

In [ ]:
# Setup and check pre-requisites to create Database
%store -r ingest_create_athena_table_passed
!pip install --disable-pip-version-check -q awswrangler==2.3.0
import awswrangler as wr


In [2]:
# Read in Datasets

In [3]:
# Dataset Info

# Data Exploration

In [4]:
# Dataset Descriptives

In [5]:
# Data Distributions

In [ ]:
# Filtering Text by Location